In [1]:
#Weekly scoring using model saved to disk and updated scoring dataset

In [2]:
#Importing basic packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from numpy import mean
from numpy import std
from matplotlib import pyplot
from numpy import set_printoptions
from pickle import dump
from pickle import load
import shap

In [3]:
#Import data for scoring
scoring = pd.read_csv('C:\\Users\\D648007\\AFL2022ScoringData.csv')


In [4]:
#View first 15 rows to see what the data is like


scoring.head(15)

,round,home_team,away_team,hm_PWk_lad,aw_PWk_lad,PWk_lad_gap,hm_PWk_perc,aw_PWk_perc,PWk_perc_gap,hm_PWk_pts,...,Pwk_win,2Pwk-win,3Pwk_win,4Pwk_win,Unnamed: 98,Unnamed: 99,HM odds,Away odds,Win Odds,Win returns
0,1,ME,WB,1,5,4,130.8,132.8,-2.0,70.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,CA,RI,13,12,-1,88.5,97.9,-9.4,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,SK,CW,10,17,7,91.5,85.6,5.9,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,GE,ES,3,8,5,126.7,109.1,17.6,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,GW,SY,7,6,-1,99.7,119.9,-20.2,46.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1,BL,PA,4,2,-2,133.3,126.3,7.0,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1,HW,NM,14,18,4,85.2,70.3,14.9,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1,AD,FR,15,11,-4,82.0,86.5,-4.5,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1,WC,GC,9,16,7,93.2,76.8,16.4,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2,WB,CA,5,13,8,132.8,88.5,44.3,60.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#Create scoring dataset of features identified through feature importance testing
scoring = scoring[["round", "hm_PWk_lad", "aw_PWk_lad", "PWk_lad_gap", "hm_PWk_perc", "aw_PWk_perc" , "PWk_perc_gap", "hm_PWk_pts", 
                  "aw_PWk_pts", "PWk_pts_gap", "hm_PWk_pts_stan", "aw_PWk_pts_stan", "PWk_pts_gap_stan", "hm_PY" , "hm_PY_perc",
                  "aw_PY", "aw_PY_perc", "PY_gap", "PY_%_gap", "2PY_%_gap", "3PY_%_gap"]]

In [6]:
#Separate the required columns into an array file for scoring by the model
array = scoring.values

score = array[:, :]


In [7]:
print(score.shape)

(198, 21)


In [8]:
#Before splitting into an array for model fitting, save the feature names for later use
rnd = score[:, 0]

In [9]:
#Rescale the scoring data 
scaler = StandardScaler()
rescaledscore=scaler.fit_transform(score)

#summarise transformed data
set_printoptions(precision=3)
print(rescaledscore[0:5, :])


[[-1.617 -1.645 -0.859  0.545  1.31   1.408 -0.053  1.622  1.024 -0.421
  -0.177 -0.355 -0.22  -1.638  1.407 -0.867  1.503 -0.549 -0.069  0.037
  -1.128]
 [-1.617  0.661  0.495 -0.116 -0.646 -0.23  -0.291 -0.59  -0.27   0.223
  -0.813 -0.675  0.035  0.675 -0.657  0.482 -0.2    0.137 -0.325 -1.161
  -1.154]
 [-1.617  0.084  1.462  0.941 -0.507 -0.807  0.2   -0.124 -1.094 -0.663
  -0.679 -0.878 -0.316  0.096 -0.511  1.446 -0.801 -0.961  0.204  0.22
  -1.336]
 [-1.617 -1.26  -0.278  0.677  1.12   0.296  0.575  1.273  0.083 -0.824
  -0.278 -0.588 -0.38  -1.253  1.206 -0.289  0.346 -0.686  0.609  1.885
   1.59 ]
 [-1.617 -0.492 -0.665 -0.116 -0.128  0.803 -0.637  0.225  1.024  0.545
  -0.579 -0.355  0.162 -0.482 -0.112 -0.675  0.874  0.137 -0.699  0.424
   0.698]]


In [10]:
#load the model afl2022model.sav from disk

filename = 'afl2022modelRF.sav'
afl2022model = load(open(filename, 'rb'))

In [11]:
#Score the scoring dataset with the model
afl2022_proba = afl2022model.predict_proba(rescaledscore)
print(afl2022_proba)

[[0.442 0.558]
 [0.544 0.456]
 [0.306 0.694]
 [0.299 0.701]
 [0.56  0.44 ]
 [0.378 0.622]
 [0.286 0.714]
 [0.564 0.436]
 [0.234 0.766]
 [0.105 0.895]
 [0.518 0.482]
 [0.41  0.59 ]
 [0.653 0.347]
 [0.08  0.92 ]
 [0.833 0.167]
 [0.804 0.196]
 [0.502 0.498]
 [0.533 0.467]
 [0.374 0.626]
 [0.257 0.743]
 [0.782 0.218]
 [0.184 0.816]
 [0.814 0.186]
 [0.072 0.928]
 [0.41  0.59 ]
 [0.479 0.521]
 [0.358 0.642]
 [0.347 0.653]
 [0.404 0.596]
 [0.079 0.921]
 [0.624 0.376]
 [0.67  0.33 ]
 [0.638 0.362]
 [0.195 0.805]
 [0.574 0.426]
 [0.681 0.319]
 [0.252 0.748]
 [0.857 0.143]
 [0.75  0.25 ]
 [0.328 0.672]
 [0.689 0.311]
 [0.184 0.816]
 [0.489 0.511]
 [0.593 0.407]
 [0.705 0.295]
 [0.507 0.493]
 [0.135 0.865]
 [0.118 0.882]
 [0.372 0.628]
 [0.864 0.136]
 [0.646 0.354]
 [0.545 0.455]
 [0.688 0.312]
 [0.55  0.45 ]
 [0.663 0.337]
 [0.232 0.768]
 [0.467 0.533]
 [0.081 0.919]
 [0.521 0.479]
 [0.134 0.866]
 [0.297 0.703]
 [0.208 0.792]
 [0.471 0.529]
 [0.403 0.597]
 [0.122 0.878]
 [0.375 0.625]
 [0.109 0.

In [12]:
##To export as a csv, where prediction is the array file with scores

prediction = pd.DataFrame(afl2022_proba).to_csv('prediction23RF.csv')

In [ ]:
#SHAP Values
# Fits the explainer onto the x-test data that it is predicting for 
explainer = shap.Explainer(afl2022model.predict, rescaledscore)
# Calculates the SHAP values - It takes some time
shap_values = explainer(rescaledscore)

Permutation explainer:  83%|███████████████████████████████████████████████▏         | 164/198 [07:59<01:41,  2.98s/it]

In [ ]:
#Shap values at an individual prediction level
#for observation [0]
shap.plots.bar(shap_values[175])

In [ ]:
#SHAP values at an individual prediction level
#Waterfall, showing contributors of gap between prediction and expected (expected = mean of all predictions)
#For observation [0]
shap.plots.waterfall(shap_values[175])
